In [82]:
# Importa las librerias a usar
from pathlib import Path
import pandas as pd
import re
from datetime import datetime
import numpy as np

In [38]:
# Definir la ruta de la carpeta en el escritorio
folder = Path('D:/AÑO 2024/REPORTES 37/JULIO')

# Obtener el nombre del mes de la carpeta
mes = folder.name.upper()

# Obtener el año de la carpeta anterior
año = folder.parent.parent.name.split()[-1]  # Asume que el año es la última palabra en el nombre de la carpeta

# Lista para almacenar todos los dataframes
all_dataframes = []

# Iterar sobre todos los archivos xlsx en la carpeta
for file in folder.glob('*.xlsx'):
    # Extraer el número del nombre del archivo
    file_number = re.search(r'\d+', file.stem)
    if file_number:
        file_number = file_number.group()
    else:
        file_number = 'N/A'  # Si no se encuentra un número

    try:
        # Leer específicamente la hoja "REPORTE37" del archivo Excel
        df = pd.read_excel(file, sheet_name='REPORTE37')
        
        # Añadir una nueva columna con el número del archivo
        df['day'] = file_number
        df['month'] = mes
        df['year'] = año
        
        # Añadir el dataframe a la lista
        all_dataframes.append(df)
        print(f"Archivo leído con éxito: {file.name}")
    except Exception as e:
        print(f"No se pudo leer la hoja 'REPORTE37' en el archivo {file.name}: {e}")

# Combinar todos los DataFrames en uno solo
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"Se han combinado {len(all_dataframes)} archivos en un DataFrame de {combined_df.shape[0]} filas y {combined_df.shape[1]} columnas.")
else:
    print("No se pudo leer ningún archivo correctamente.")


Archivo leído con éxito: Informe Morosidad ACR 10 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 11 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 12 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 15 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 16 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 17 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 18 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 19 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 2 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 22 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 23 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 24 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 25 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 26 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 29 JULIO.xlsx
Archivo leído con éxito: Informe Morosidad ACR 3 JULIO.xlsx
Archivo leído con éxito: I

In [39]:
combined_df_copy = combined_df.copy()

In [52]:
# Diccionario para mapear los nombres de los meses a números
meses_a_numeros = {
    "ENERO": 1,
    "FEBRERO": 2,
    "MARZO": 3,
    "ABRIL": 4,
    "MAYO": 5,
    "JUNIO": 6,
    "JULIO": 7,
    "AGOSTO": 8,
    "SEPTIEMBRE": 9,
    "OCTUBRE": 10,
    "NOVIEMBRE": 11,
    "DICIEMBRE": 12
}

# Crear una nueva columna 'Mes_numero' con los valores numéricos de los meses
combined_df_copy['Mes_numero'] = combined_df_copy['Mes_archivo'].map(meses_a_numeros)
combined_df_copy['Año_archivo'] = combined_df_copy['Año_archivo'].astype(int)
combined_df_copy['Numero_Archivo2'] = pd.to_numeric(combined_df_copy['Numero_Archivo'])
# Renombrar las columnas temporalmente para crear la columna 'Fecha'
combined_df_copy = combined_df_copy.rename(columns={'Año_archivo': 'year', 'Mes_numero': 'month', 'Numero_Archivo2': 'day'})
# Crear la nueva columna 'Fecha'
combined_df_copy['Fecha'] = pd.to_datetime(combined_df_copy[['year', 'month', 'day']])
# Calcular la mora en días
combined_df_copy['Dias_mora_calculado'] = (combined_df_copy['Fecha'] - combined_df_copy['Fch Vencim Cuota'] ).dt.days
# Definir los rangos y los valores correspondientes
condiciones = [
    (combined_df_copy['Dias_mora_calculado'] < 6),
    (combined_df_copy['Dias_mora_calculado'] < 11),
    (combined_df_copy['Dias_mora_calculado'] < 31),
    (combined_df_copy['Dias_mora_calculado'] < 61),
    (combined_df_copy['Dias_mora_calculado'] < 91),
    (combined_df_copy['Dias_mora_calculado'] < 121),
    (combined_df_copy['Dias_mora_calculado'] < 361)
]

valores = [
    0,
    (combined_df_copy['Capital Vencido'] ) * 0.071,
    (combined_df_copy['Capital Vencido'] ) * 0.095,
    (combined_df_copy['Capital Vencido']) * 0.119,
    (combined_df_copy['Capital Vencido'] ) * 0.155,
    (combined_df_copy['Capital Vencido'] ) * 0.167,
    (combined_df_copy['Capital Vencido'] ) * 0.238
]

# Aplicar las condiciones para calcular 'Gastos_de_cobranza_calculados'
combined_df_copy['Gastos_de_cobranza_calculados'] = np.select(condiciones, valores, default=(combined_df_copy['Capital Vencido']) * 0.298)
# Redondear a dos decimales
combined_df_copy['Gastos_de_cobranza_calculados'] = combined_df_copy['Gastos_de_cobranza_calculados'].round(2)


In [91]:
combined_df_copy.head()

,No Crédito,Cedula,Apellidos,Nombres,Direccion,Barrio,CIUDAD1,Tel. Fijo,Celular,Cupo,...,Dias Mora,Valor Gastos de Cobranza,Numero_Archivo,Mes_archivo,year,month,day,Fecha,Dias_mora_calculado,Gastos_de_cobranza_calculados
0,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,1112,5458.55540,10,JULIO,2024,7,10,2024-07-10,1086,5458.56
1,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,1081,5567.49228,10,JULIO,2024,7,10,2024-07-10,1055,5567.49
2,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,1050,5678.59562,10,JULIO,2024,7,10,2024-07-10,1024,5678.60
3,1100010,1007348023,SEPULVEDA LOTERO ...,YEFERSON ...,ALEJO-marles@hotmail.com,VARIANTE,Cartago,2095000,3147727566,1,...,1020,5791.93396,10,JULIO,2024,7,10,2024-07-10,994,5791.93
4,1100026,1235048274,AGUILAR RODRIGUEZ ...,RAMON ENRIQUE ...,CLL 110 # 91 308,NaN,Barranquilla,NaN,3164529173,1,...,1354,528.29738,10,JULIO,2024,7,10,2024-07-10,1328,528.30


In [97]:
# Primero, agrupamos por 'No Crédito' para obtener valores únicos de 'Valor Compra' e 'Intereses de Mora'
df_credito = combined_df_copy.groupby(['Fecha','No Crédito']).agg({
    'Cedula': 'first',
    'Valor Compra': 'first',
    'Intereses de Mora': 'first',
    'Vlr. Cuota Vencida': 'sum',
    'Intereses Vencidos': 'sum',
    'Valor Gastos de Cobranza': 'sum',
    'Capital Vencido': 'sum',
    'Gastos_de_cobranza_calculados': 'sum'  
    
}).reset_index()




In [98]:
df_credito_sum = df_credito.groupby(['Fecha','Cedula']).agg({
    'Valor Compra': 'sum',
    'Intereses de Mora': 'sum',
    'Vlr. Cuota Vencida': 'sum',
    'Intereses Vencidos': 'sum',
    'Valor Gastos de Cobranza': 'sum',
    'Capital Vencido': 'sum',
    'Gastos_de_cobranza_calculados': 'sum'
}).reset_index()


In [99]:
df_credito_sum

,Fecha,Cedula,Valor Compra,Intereses de Mora,Vlr. Cuota Vencida,Intereses Vencidos,Valor Gastos de Cobranza,Capital Vencido,Gastos_de_cobranza_calculados
0,2024-07-02,334993,800000,851.03,164495.22,19712.00,14974.77198,125838.42,11954.65
1,2024-07-02,521570,1805000,638452.64,1935682.10,155420.86,447198.53952,1500666.24,439587.84
2,2024-07-02,546127,404600,558.30,98919.83,6783.39,9824.05809,82555.11,7842.74
3,2024-07-02,582095,975000,328647.83,1407123.24,188233.80,268890.82980,975000.00,263834.89
4,2024-07-02,629770,217000,89225.55,173487.16,6064.80,46766.64788,156935.06,46766.64
...,...,...,...,...,...,...,...,...,...
383428,2024-07-31,1243139989,1041500,561330.61,819324.13,47185.10,199939.50314,670937.93,199939.50
383429,2024-07-31,2000001098,500000,69677.00,578822.77,33322.77,149000.00000,500000.00,149000.00
383430,2024-07-31,2000002521,840000,2315.52,205477.02,26268.14,14910.35133,139424.80,14896.22
383431,2024-07-31,2000004082,1125000,76290.15,866928.57,65950.19,141933.91167,654274.56,141933.93
